In [11]:
!pip install q kaggle
!pip install snorkel
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [1]:
! mkdir ~/.kaggle

In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle datasets download mananjainb21ai021/financial-news-data

  0% 0.00/2.38M [00:00<?, ?B/s]
100% 2.38M/2.38M [00:00<00:00, 182MB/s]


In [9]:
! unzip financial-news-data

Archive:  financial-news-data.zip
  inflating: finance_data.csv        


In [12]:
from google.colab import files
import io
import pandas as pd
#Snorkel
from snorkel.labeling import LabelingFunction
import re
from snorkel.preprocess import preprocessor
from textblob import TextBlob
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function
#NLP packages
import spacy
from nltk.corpus import stopwords
import string
import nltk
import nltk.tokenize
import matplotlib.pyplot as plt
punc = string.punctuation
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
#Supervised learning
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
##Deep learning libraries and APIs
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
data=pd.read_csv("finance_data.csv")
# data=data.drop(["Unnamed: 5","URL"],axis=1)
# data['Date']=pd.to_datetime(data['Date'])
data.head()

,Date,Title,URL,sentiment,confidence,Text
0,2017-01-03,​Meenakshi Dawar and Varun Goel join Reliance ...,http://economictimes.indiatimes.com/mf/mf-news...,NEGATIVE,-0.838119,Get ready for some fund manager churning. Meen...
1,2017-01-03,"Will follow Reliance Jio pricing , says Bharti...",http://www.financialexpress.com/industry/will-...,NEGATIVE,-0.925142,The Financial ExpressBharti Airtel chairman Su...
2,2017-01-04,Reliance Jio Prime membership plan extended ti...,http://www.financialexpress.com/industry/techn...,POSITIVE,0.794206,The Financial ExpressReliance Jio has announce...
3,2017-01-06,"Reliance Retail : Ladies , now de",http://economictimes.indiatimes.com/industry/s...,POSITIVE,0.964752,MUMBAI: Reliance Retail is launching stores wh...
4,2017-01-06,Internet Trends Report highlights India mobile...,http://www.livemint.com/Technology/ssczejSaUSV...,NEGATIVE,-0.993098,New Delhi: The volume of wireless broadband da...


In [18]:
def get_sequences(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)

    sequences = tokenizer.texts_to_sequences(texts)
    print("Vocab length:", len(tokenizer.word_index) + 1)

    max_seq_length = np.max(list(map(lambda x: len(x), sequences)))
    print("Maximum sequence length:", max_seq_length)

    sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

    return sequences


In [19]:
def preprocess_inputs(df):
    df = df.copy()

    sequences = get_sequences(df['Title'])

    label_mapping = {
        'NEGATIVE': 0,
        'POSITIVE': 1
    }

    y = df['sentiment'].replace(label_mapping)

    train_sequences, test_sequences, y_train, y_test = train_test_split(sequences, y, train_size=0.7, shuffle=True, random_state=1)

    return train_sequences, test_sequences, y_train, y_test


In [20]:
train_sequences, test_sequences, y_train, y_test = preprocess_inputs(data)

Vocab length: 3818
Maximum sequence length: 25


In [21]:
train_sequences


array([[  67,   17,   89, ...,    0,    0,    0],
       [   6,    5,   40, ...,    0,    0,    0],
       [3513,   10,  219, ...,    0,    0,    0],
       ...,
       [   1,   23,   19, ...,    0,    0,    0],
       [   1,    4,    1, ...,    0,    0,    0],
       [   4, 1067,   17, ...,    0,    0,    0]], dtype=int32)

In [22]:
y_train


750     1
1330    1
1482    0
409     1
12      1
       ..
715     1
905     0
1096    1
235     1
1061    1
Name: sentiment, Length: 1196, dtype: int64

In [23]:
inputs = tf.keras.Input(shape=(train_sequences.shape[1],))
x = tf.keras.layers.Embedding(
    input_dim=48315,
    output_dim=128,
    input_length=train_sequences.shape[1]
)(inputs)
x = tf.keras.layers.GRU(256, return_sequences=True, activation='tanh')(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_sequences,
    y_train,
    validation_split=0.3,
    batch_size=16,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)


Epoch 1/100
53/53 [==============================] - 7s 72ms/step - loss: 0.6850 - accuracy: 0.5484 - val_loss: 0.6610 - val_accuracy: 0.5822
Epoch 2/100
53/53 [==============================] - 2s 31ms/step - loss: 0.3780 - accuracy: 0.8303 - val_loss: 0.7124 - val_accuracy: 0.6323
Epoch 3/100
53/53 [==============================] - 1s 16ms/step - loss: 0.0704 - accuracy: 0.9797 - val_loss: 0.9586 - val_accuracy: 0.6490
Epoch 4/100
53/53 [==============================] - 1s 16ms/step - loss: 0.0259 - accuracy: 0.9964 - val_loss: 0.9840 - val_accuracy: 0.6435


In [24]:
model.evaluate(test_sequences,y_test)

17/17 [==============================] - 0s 4ms/step - loss: 0.6785 - accuracy: 0.5575


[0.678493082523346, 0.5575048923492432]

In [25]:
y_test.value_counts()

sentiment
0    268
1    245
Name: count, dtype: int64

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25)]              0         
                                                                 
 embedding (Embedding)       (None, 25, 128)           6184320   
                                                                 
 gru (GRU)                   (None, 25, 256)           296448    
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 2)                 12802     
                                                                 
Total params: 6493570 (24.77 MB)
Trainable params: 6493570 (24.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
